In [1]:
cd ..

/mnt/ceph/users/zzhang/CRISPR_pred/crispr_kinn


In [2]:
import pandas as pd
from tqdm import tqdm
from Bio import pairwise2
from Bio.Seq import Seq
from src.crispr_kinn_predict import reload_from_dir, KineticNeuralNetworkBuilder, \
    featurize_alignment, get_letter_index
import tensorflow as tf
import numpy as np

Using TensorFlow backend.


In [3]:
gene = pd.read_table("/mnt/ceph/users/zzhang/CRISPR_pred/crispr_kinn/VEGFA.bed",
                    names=['chrom', 'start', 'end', 'strand', 'PamID', 'genename', 'num', 'seq'])

In [4]:
gene.head(3)

,chrom,start,end,strand,PamID,genename,num,seq
0,6,43770677,43770737,+,VEGFA|1,VEGFA,1,CCTTGGGATCCCGCAGCTGACCAGTCGCGCTGACGGACAGACAGAC...
1,6,43770704,43770764,-,VEGFA|2,VEGFA,2,GGCCGGGGAGGAGGTGGTAGCTGGGGCTGGGGGCGGTGTCTGTCTG...
2,6,43770707,43770767,-,VEGFA|3,VEGFA,3,GCCGGCCGGGGAGGAGGTGGTAGCTGGGGCTGGGGGCGGTGTCTGT...


In [5]:
grna_seq = []
for i in range(gene.shape[0]):
    # 20 gRNA + 3 PAM
    grna_seq.append(gene.iloc[i]['seq'][13:36])
gene['grna_seq'] = grna_seq

In [6]:
# sanity check
for _ in grna_seq:
    assert _.endswith('GG')

In [7]:
gene.shape

(261, 9)

In [8]:
n_mismatch = 6
n_bulges = 1
with open("CasOffinder_input.txt", "w") as f:
    # first line: FASTA filepath
    f.write("/mnt/ceph/users/zzhang/genome_assembly/GRCh38.primary_assembly.genome.fa\n")
    # second line: desired pattern including PAM site and optional DNA or RNA bulge sizes, separated by spaces
    f.write(f"{'N'*20}NGG {n_bulges} {n_bulges}\n")
    # remaining lines: the query sequences and maximum mismatch numbers, separated by spaces
    for i in range(gene.shape[0]):
        f.write(f"{gene.iloc[i]['grna_seq']} {n_mismatch}\n")

In [9]:
%%time
%%bash
cas-offinder CasOffinder_input.txt C CasOffinder_output.txt

Total 1 device(s) found.
Loading input file...
Reading /mnt/ceph/users/zzhang/genome_assembly/GRCh38.primary_assembly.genome.fa...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
549.912 seconds elapsed.


CPU times: user 11.2 ms, sys: 14.9 ms, total: 26.2 ms
Wall time: 9min 9s


In [10]:
cas_off_ver = '2.4'

if cas_off_ver == '2.4':
    ot = pd.read_table("./notebooks/CasOffinder_output.txt",
                      names=['grna_seq', "chrom", "pos", "ot_seq", "strand", "mm"])
    ot['chrom'] = [x.split()[0] for x in ot['chrom']]
    ot = ot.query('mm>0').reset_index()
else:
    ot = pd.read_table("./notebooks/CasOffinder_output.head.txt", skiprows=1, low_memory=False)
    ot = ot.query('Mismatches>0').reset_index() #for V3.0
    # fix bulge
    for i, row in tqdm(ot.iterrows(), total=ot.shape[0]):
        if row['crRNA'].startswith('-'):
            ot.iloc[i]['crRNA'] = ot.iloc[i]['crRNA'].lstrip('-')
            ot.iloc[i]['DNA'] = ot.iloc[i]['DNA'][1:]
        if row['crRNA'].endswith('-'):
            ot.iloc[i]['crRNA'] = ot.iloc[i]['crRNA'].lstrip('-')
            ot.iloc[i]['DNA'] = ot.iloc[i]['DNA'][:-1]

In [11]:
print(ot.shape)
ot.head()

(414893, 7)


,index,grna_seq,chrom,pos,ot_seq,strand,mm
0,0,CAGCTGACCAGTCGCGCTGACGG,chr5,173177545,CAaaTGACCAGTCtgGCTGAgGGCCCT,-,5
1,1,CAGCTGACCAGTCGCGCTGACGG,chr3,194615410,CAGCTGtCCtGTCtCGCTGAgGGAATA,-,4
2,2,CAGCTGACCAGTCGCGCTGACGG,chr6,35404622,CAGCTGACCAGgaGCaCaGAgGGCTGC,+,5
3,3,CAGCTGACCAGTCGCGCTGACGG,chr7,571289,CAGCTGACCccaCcCGaTGACGGGGTG,+,5
4,4,CAGCTGACCAGTCGCGCTGACGG,chr4,601716,CtaCTcACCAGTgtCGCTGACGGGAGT,+,5


In [12]:
if cas_off_ver == '2.4':
    ### if using Cas-OFFinder v2.4
    alignments = []
    maxlen = 25
    for _, row in tqdm(ot.iterrows(), total=ot.shape[0]):
        ref = Seq(row['grna_seq'][::-1])
        alt = Seq(row['ot_seq'].upper()[0:23][::-1])
        # m: A match score is the score of identical chars, otherwise mismatch score
        # d: The sequences have different open and extend gap penalties.
        aln = pairwise2.align.localxd(ref, alt, -1, -0.1, -1, 0)
        if len(aln[0][0]) > maxlen: # increase gap open penalty to avoid too many gaps
            aln = pairwise2.align.localxd(ref, alt, -5, -0.1, -5, 0)
            if len(aln[0][0]) > maxlen:
                aln = [(ref, alt)]
        alignments.append(aln[0])
    alignment_df = pd.DataFrame({'ref':[x[0] for x in alignments], 'alt':[x[1] for x in alignments]})
elif cas_off_ver == '3.0':
    ### if using Cas-OFFinder v3.0
    alignments = [x[1].str[::-1].str.upper().tolist() for x in tqdm(ot[['crRNA', 'DNA']].iterrows(), total=ot.shape[0])]
    alignment_df = pd.DataFrame({'ref':[x[0] for x in alignments], 'alt':[x[1] for x in alignments]})


100%|██████████| 414893/414893 [03:37<00:00, 1904.57it/s]


In [13]:
# featurize alignment
ltidx = get_letter_index(build_indel=True)
fea = featurize_alignment(alignments, ltidx)

In [14]:
# load kinn
sess = tf.Session()
manager_kwargs={
    'output_op': 
        lambda: tf.keras.layers.Lambda(lambda x: tf.math.log(x)/np.log(10), name="output_log"),
    'n_feats': 25,  # remember to change this!!
    'n_channels': 9,
    'batch_size': 128,
    'epochs': 30,
    'earlystop': 10,
    'verbose': 0
}
kinn_1 = reload_from_dir(wd="outputs/2022-05-21/KINN-wtCas9_cleave_rate_log-finkelstein-0-rep4-gRNA1/",
                         sess=sess,
                         manager_kwargs=manager_kwargs,
                         model_fn=KineticNeuralNetworkBuilder
                         )
kinn_2 = reload_from_dir(wd="outputs/2022-05-21/KINN-wtCas9_cleave_rate_log-finkelstein-0-rep5-gRNA2/",
                         sess=sess,
                         manager_kwargs=manager_kwargs,
                         model_fn=KineticNeuralNetworkBuilder
                         )

loaded searched model
loaded searched model


In [15]:
# build rate model
l1 = {l.name:l for l in kinn_1.model.layers}
kinn_1_rate_mod = tf.keras.Model(
    inputs=kinn_1.model.inputs,
    outputs=l1['gather_rates'].output
)
l2 = {l.name:l for l in kinn_2.model.layers}
kinn_2_rate_mod = tf.keras.Model(
    inputs=kinn_2.model.inputs,
    outputs=l2['gather_rates'].output
)

In [16]:
k1_clv = kinn_1.predict(fea)
k2_clv = kinn_2.predict(fea)
k1_rates = np.array(kinn_1_rate_mod.predict(kinn_1.blockify_seq_ohe(fea)))
k2_rates = np.array(kinn_2_rate_mod.predict(kinn_2.blockify_seq_ohe(fea)))

In [17]:
k1_kinn = pd.DataFrame(np.hstack([k1_clv, k1_rates]), 
                       columns=['pred_cleavage_log10', 'k_on_log', 'k_off_log', 'k_OI_log', 
                                'k_IO_log', 'k_IC_log', 'k_CI_log', 'k_cat_log'])
k2_kinn = pd.DataFrame(np.hstack([k2_clv, k2_rates]), 
                       columns=['pred_cleavage_log10', 'k_on_log', 'k_off_log', 'k_OI_log', 
                                'k_IO_log', 'k_IC_log', 'k_CI_log', 'k_cat_log'])
kinn = k1_kinn.join(k2_kinn, lsuffix='.1', rsuffix='.2')

In [18]:
ot_res = ot.join(alignment_df).join(kinn)

In [19]:
ot_res['pred_cleavage_log10'] = ot_res[['pred_cleavage_log10.1', 'pred_cleavage_log10.2']].mean(axis=1)

In [20]:
ot_res.shape

(414893, 26)

In [21]:
#row = ot_res.sort_values("pred_cleavage_log10", ascending=False).iloc[0]
#row

In [22]:
ot_res = ot_res.query('pred_cleavage_log10 > -8')

In [23]:
ot_res_w_on = ot_res.set_index('grna_seq').join(gene.set_index('grna_seq'), lsuffix='.off', rsuffix='.on').reset_index()

In [24]:
ot_res_w_on = ot_res_w_on[[
    'PamID', 'chrom.on', 'start', 'end', 'strand.on', 'genename', 'grna_seq', 'ot_seq', 'mm',
    'chrom.off', 'pos', 'strand.off', 'ref', 'alt', 'pred_cleavage_log10', 'pred_cleavage_log10.1', 'pred_cleavage_log10.2',
    'k_on_log.1', 'k_off_log.1', 'k_OI_log.1', 'k_IO_log.1', 'k_IC_log.1', 'k_CI_log.1', 'k_cat_log.1',
    'k_on_log.2', 'k_off_log.2', 'k_OI_log.2', 'k_IO_log.2', 'k_IC_log.2', 'k_CI_log.2', 'k_cat_log.2',
]]

In [25]:
ot_res_w_on.query("PamID=='VEGFA|1'").sort_values('pred_cleavage_log10', ascending=False)[['ref', 'alt', 'pred_cleavage_log10']].head()

,ref,alt,pred_cleavage_log10
43518,GGCAGTCGCGCTGACCAGTCGAC,GGGAGTCCCACTGACCAGTCGAA,-3.771010
43503,GGCAGTCGCGCTGACCAGTCGAC,GGGAGTCGGTCTGACCAGTAAAC,-4.689523
43520,GGCAGTCGCGCTGACCAGTCGAC,GGGAGTAGAGGTGACCAGTCGTC,-4.892593
43538,GGCAGTCGCGCTGACCAGTCGAC,GGTAGTCACGTAGACCAGTCAAC,-5.441313
43517,GGCAGTCGCGCTGACCAGTCGAC,GGGAGTCGCACTGACCTGCCGAG,-5.602283


In [26]:
ot_res_w_on.query("PamID=='VEGFA|2'").sort_values('pred_cleavage_log10', ascending=False)[['ref', 'alt', 'pred_cleavage_log10']].head()

,ref,alt,pred_cleavage_log10
178711,GGCGGGGGTCGGGGTCGATGGTG,GGAGGGGGTCGGGGTCGATGGGG,-1.287670
178989,GGCGGGGGTCGGGGTCGATGGTG,GGAGGGGGTCGGGGTCGTTGGTG,-1.983200
179838,GGCGGGGGTCGGGGTCGATGGTG,GGTGGGGGTAGGGGTCGATGGTG,-2.136131
179755,GGCGGGGGTCGGGGTCGATGGTG,GGAGGGGGTCGGAGTCGATAGAG,-2.214801
179502,GGCGGGGGTCGGGGTCGATGGTG,GGGGGTGGTCGGGGTCGATGGTT,-2.261375


In [27]:
ot_res.to_csv("VEGFA-KINN_pred.txt", sep="\t", index=False)

In [28]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Mon Aug 29 2022

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.22.0

numpy     : 1.21.6
pandas    : 1.3.5
tensorflow: 1.15.0
Bio       : 1.79

Watermark: 2.3.1



In [29]:
%%bash
cas-offinder

Cas-OFFinder v2.4 (Mar  5 2018)

Copyright (c) 2013 Jeongbin Park and Sangsu Bae
Website: http://github.com/snugel/cas-offinder

Usage: cas-offinder {input_file} {C|G|A}[device_id(s)] {output_file}
(C: using CPUs, G: using GPUs, A: using accelerators)

Example input file:
/var/chromosomes/human_hg19
NNNNNNNNNNNNNNNNNNNNNRG
GGCCGACCTGTCGCTGACGCNNN 5
CGCCAGCGTCAGCGACAGGTNNN 5
ACGGCGCCAGCGTCAGCGACNNN 5
GTCGCTGACGCTGGCGCCGTNNN 5

Available device list:
Type: CPU, ID: 0, <pthread-Intel(R) Xeon(R) Gold 6148 CPU @ 2.40GHz> on <Portable Computing Language>
